In [9]:
import os
import numpy as np
import csv
import pandas as pd

dataDir = '/dcs/project/seal-script-project-data/seal-script-new/scraped/'
charListPath = './source/classical_chars.csv'
filetype = 'png'

In [2]:
def load_data(char_from, char_to):
    data = np.genfromtxt(os.path.join(dataDir, 'hsk.csv'), delimiter=',', encoding='utf8', dtype=None)
    char_array = data[char_from-1:char_to]
    print(f"Imported {char_array.shape[0]} characters from index {char_from} to {char_to}")
    
    return char_array

def checkMissingFolders():
    missing_chars = []
    charList = pd.read_csv(charListPath, names=['idx', 'char'])
    for i, (idx, char) in charList.iterrows():
        charDir = os.path.join(dataDir, str(idx))
        # Get list of all objects in directory
        files = os.listdir(charDir)
        # Keep objects which are files
        files = [f for f in files if os.path.isfile(charDir+'/'+f) and f'.{filetype}' in f]
        # Get number of files in directory
        imageCount = len(files)

        if imageCount == 0:
            missing_chars.append(idx)
            
    return missing_chars

def save_missing_chars(data):
    with open(os.path.join(dataDir, "missing_chars.csv"),"w+", encoding="utf8", newline="") as f:
        csvWriter = csv.writer(f)
        csvWriter.writerows(data)
        
        f.close()

### Check that all characters in the main store are valid 

In [37]:
modern_char_store = './source/hsk.csv'

def check_modern_char_store(dir):
    # Load in the data
    data = np.genfromtxt(dir, delimiter=',', encoding='utf8', dtype=None)
    print(data.dtype)
    valid_chars = []

    for i in range(data.shape[0]):

        # Add checks here
        if data[i][1] == '\ufeff':
            pass
        else: # If the character is valid
            valid_chars.append(data[i][1])
    
    formatted_data = np.asarray([(i+1, valid_chars[i]) for i in range(len(valid_chars))], dtype=[('f0', '<i8'), ('f1', '<U1')])
    
    print(formatted_data[417])
    print(formatted_data.shape)
    print(formatted_data)

    # Save the new formatted data to file
    np.savetxt('./source/hsk.csv', formatted_data, delimiter=',', fmt='%s')
    


check_modern_char_store(modern_char_store)

[('f0', '<i8'), ('f1', '<U1')]
(418, '根')
(1075,)
[(   1, '起') (   2, '果') (   3, '热') ... (1073, '赢') (1074, '骗')
 (1075, '阅')]


### Relabel image folders and image filenames

In [69]:
# Shift folder ID down by 1: 420->419, 421->420 etc.

dir = '/dcs/project/seal-script-project-data/seal-script-images-raw-copy/'

dirs = [str(i) for i in range(420,1076)]
print(dirs)
for i in range(len(dirs)):
    os.rename(dir+dirs[i], dir+str(int(dirs[i])-1))




# for root,files,names in os.walk(dir):
#     print(root)
#     print(names)
#     if os.path.isdir(root) and root.split('/')[-1] != '':
#         if int(root.split('/')[-1]) >= 420:
#             char_id = root.split('/')[-1]
#             print(dir+str(int(char_id)-1))
#             os.rename(root, dir+str(int(char_id)-1))
#             # if char_id == '419':
#             #     print()
#             # for idx, name in enumerate(names):
#                 # print(char_id+"_"+str(idx+1)+".png")
#                 # os.rename(os.path.join(root, name), os.path.join(root, char_id+"_"+str(idx+1)+".png"))
            

['420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470', '471', '472', '473', '474', '475', '476', '477', '478', '479', '480', '481', '482', '483', '484', '485', '486', '487', '488', '489', '490', '491', '492', '493', '494', '495', '496', '497', '498', '499', '500', '501', '502', '503', '504', '505', '506', '507', '508', '509', '510', '511', '512', '513', '514', '515', '516', '517', '518', '519', '520', '521', '522', '523', '524', '525', '526', '527', '528', '529', '530', '531', '532', '533', '534', '535', '536', '537', '538', '539', '540', '541', '542', '543', '544', '545', '546', '547', '548', '549', '550', '551', '552', '553', '554', '555', '556', '557', '558', '559', '560', '561', '562'

FileNotFoundError: [Errno 2] No such file or directory: '/dcs/project/seal-script-project-data/seal-script-images-raw-copy/1075'

### Delete all images

In [12]:
def deleteAllImages():
    charList = pd.read_csv(charListPath, names=['idx', 'char'])
    for i, (idx, char) in charList.iterrows():
        charDir = os.path.join(dataDir, str(idx))
  
        for filename in os.listdir(charDir):
            file_path = os.path.join(charDir, filename)
            
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

# deleteAllImages()

In [72]:


dirs = [str(i) for i in range(418,1076)]

for i in range(len(dirs)):
    files = os.listdir(dir+dirs[i])
    for j in range(len(files)):
        os.rename(dir+dirs[i]+"/"+files[j], dir+dirs[i]+"/"+dirs[i]+"_"+str(j+1)+".png")

In [17]:
def main():
    missed_chars = check_missing_folders(load_data(1, 1076))
    print('Characters without images:')
    print(missed_chars)
    print(f'Found {len(missed_chars)} characters without images')
    save_missing_chars(missed_chars)

In [18]:
main()

Imported 1076 characters from index 1 to 1076
Characters without images:
[['汽', '210'], ['踢', '215'], ['睛', '312'], ['参', '424'], ['辆', '494'], ['裤', '604'], ['乓', '685'], ['熟', '715'], ['忆', '771'], ['擦', '775'], ['尊', '789'], ['住', '87'], ['钥', '906'], ['够', '958']]
Found 14 characters without images
